# Constituer un corpus de recherche avec une API

## [Notebook précédent](https://colab.research.google.com/drive/1NHCg4HiTZVZbeZqeSMJthFw6r5TYzqCB?usp=sharing)

---

## Introduction

Comme on aime bien faire les choses à l'envers, on va ici créer le corpus de recherche qui a été utilisé pour produire les visualisations dans le notebook précédent. La chaîne de traitement est la suivante:
- les auteur.ice.s sont définis pour chaque genre
- tous les manuscrits mis en vente dans les catalogues Katabase et écrits par ces auteur.ice.s sont récupérés sur l'API de Katabase. Cette étape est le point central du notebook.
- les données récupérés sont nettoyées et structurées obtenir 4 corpus en texte brut
- les corpus sont enregistrés dans des fichiers

---

## Imports et installation des dépendences

On installe des dépendances et importe les librairies, comme dans la partie précédente.

In [ ]:
!pip install certifi==2022.12.7
!pip install charset-normalizer==3.0.1
!pip install idna==3.4
!pip install kaleido==0.2.1
!pip install pkg_resources==0.0.0
!pip install plotly==5.13.0
!pip install requests==2.28.2
!pip install tenacity==8.2.1
!pip install urllib3==1.26.14

In [ ]:
import requests  # le paquet pour faire des requêtes HTTP et récupérer des données de sources distantes
import json  # librairie pour le json
import sys  # librarie pour des opérations sur le système d'exploitation
import os  # le paquet pour construire des chemins de fichiers
import re  # librairie pour les expressions régulières

---

## La constitution de corpus

La première étape est de définir qui sont les auteur.ice.s à partir desquel.le.s on constituera notre corpus de recherche. Ce corpus est constitué à partir de la page Wikipédia sur la [littérature française du XVIIIe siècle](https://fr.wikipedia.org/wiki/Litt%C3%A9rature_fran%C3%A7aise_du_XVIIIe_si%C3%A8cle). 

En essayer de créer le corpus, des soucis apparaissent très vite: 
- on est très limité.e.s par les données mises à disposition par Katabase (quelles sources ont été océrisées, lesquelles ne l'ont pas été?)
- on est ensuite limité par les sources elles-mêmes: ce qui est mis en vente est ce qui a une certaine visibilité au XIXe siècle. Les autrices, par exemple (et les femmes en général), sont très sous-représentées dans le corpus 
- on est également limité.e.s par la manière dont elles sont mises à disposition: l'API ne permet de faire des requêtes sur les noms de famille (il y a donc un risque de bruit en cas d'homonymie entre plusieurs personnes).
- le dernier problème tient à notre hypothèse de recherche elle-même: un.e auteur.ice n'appartient pas à un genre, elle ou il pratique souvent différents genres. Mais on continue quand même avec cette hypothèse: le but de ce cours c'est de montrer des outils, pas de révolutionner la science. 

Par conséquent, les corpus constitués sont uniquement masculins (il y a *très* peu de données et encore moins de manuscrits de femmes vendus à prix fixes) et comportent des auteurs qui semblent ne pas avoir d'homonymes dans le corpus de Katabase, pour éviter le bruit. On classe enfin les auteurs en fonction du genre dans lequel ils sont le plus connus.

In [ ]:
auteurs_idees = [
        "voltaire"       # françois-marie arouet, dit voltaire
        , "montesquieu"  # charles louis de secondat, baron de montesquieu
        , "rousseau"     # jean-jacques rousseau
        , "diderot"      # denis diderot
        , "d'alembert"    # jean le rond d'alembert
]
auteurs_theatre = [
        "beaumarchais"  # pierre-augustin caron de beaumarchais
        , "marivaux"    # marivaux, né pierre carlet
        , "regnard"     # jean-françois regnard
        , "lesage"      # alain-rené lesage
        , "sedaine"     # michel-jean sedaine
]
auteurs_roman = [
        "restif de la bretonne"  # nicolas edme restif de la bretonne
        , "laclos"               # choderlos de laclos
        , "sade"                 # marquis de sadee
        , "crebillon"            # claude-prosper joylot de crébillon
        , "cazotte"              # jacques cazotte
]
auteurs_poeme = [
        "lefranc de pompignan"  # jean-jacques lefranc de pompignan
        , "gilbert"             # nicolas gilbert
        , "delille"             # jacques delille
        , "chenier"             # andré chénier
        , "parny"               # évariste de parny
]

# variables contenant nos 4 corpus
data_idees = {}
data_theatre = {}
data_roman = {}
data_poeme = {}

---

## Constituer le corpus via `KatAPI`, l'API de Katabase


### Les API en théorie

#### Architecture client-serveur

Pour rappel, le Web est défini selon une **architecture "[client-serveur](https://fr.wikipedia.org/wiki/Client-serveur)"**:
- **le serveur**, c'est l'infrastructure technique qui met un site internet à disposition et qui le fait fonctionner.
- **le client**, c'est ce qui utilise le site internet en interagissant avec le serveur (envoi et récupération de données).
- Pour permettre la communication entre les deux, un ensemble de standards sont définis. Ils définissent une manière pour le client et le serveur de communiquer. Le standard le plus connu est le standard HTTP. Pour faire très simple, *naviguer sur le Web, c'est poser des questions (dites *requêtes*) à des serveurs, récupérer et afficher les réponses de ces serveurs*. 

#### Une API, c'est quoi?
La manière la plus brève de définir une API Web, c'est de dire que **c'est un site internet pour machines**. Une API permet à une machine d'interagir avec un serveur à distance, de lui envoyer et de récupérer des données, de façon automatique. Une API est à la fois:
- **une interface**: elle définit une syntaxe précise pour que le client interagisse avec le serveur. C'est ce côté là qui nous importe. 
- **un service, une application** (du côté du serveur, elle traite des requêtes, récupère et envoie des données)

Dans les faits, une API diffère surtout d'un site Web "pour humain.es" parce qu'elle est pensée premièrement pour les machines: elle utilise des formats et des syntaxes simples et structurés pour que les machines puissent s'en servir facilement. La logique derrière les sites Web et les API est la même: on fait construit une URL pour faire une requête et on récupère un résultat. 
- Dans le cas d'un site normal, il s'agit d'un résultat lisible par les humain.es (*human readable*): une page HTML.
- Dans le cas d'un site, il s'agit d'un résultat structuré, *machine readable*: `json`, `xml`...

Une API est aussi utilisable *à la main*: voir par exemple [cette requête sur Madame de Sévigné](https://katabase.huma-num.fr/katapi?level=item&name=S%C3%A9vign%C3%A9&sell_date=1800-1900&orig_date=1500-1800&format=json). À l'inverse, un site Web peut être accédé par une machine, comme on va le voir.

#### Pourquoi est-ce les API c'est super pour la recherche?

Vaste question. Dans le cadre d'une recherche *en solo*, les APIs permettent d'accéder automatiquement à de très grands corpus mis en ligne par des institutions de recherche, des bibliothèques... Elles permettent souvent de faire des recherches et de récupérer des données pertinentes. 

Dans le cadre du projet Katabase, par exemple, l'API Wikidata a été utilisée pour identifier les auteur.ice.s des 100.000 manuscrits présents dans la base de données du projet. Grâce aux API Wikidata, là encore, une base de données sur mesure sur ces auteur.ices a été constituée, pour mieux comprendre la constitution du canon littéraire au XIXe siècle.

### Utiliser une API avec Python: le module `requests`

[`requests`](https://docs.python-requests.org/en/latest/index.html) est un module très simple pour faire des requêtes HTTP sur le Web. 

Le principe du HTTP est simple: 
- on définit une méthode de requête (`GET` pour récupérer des données, `POST` pour en envoyer...)
- une URL (l'endroit où l'on veut, soit envoyer, soir obtenir des données
- des paramètres supplémentaires, si besoin.

Le serveur visé par l'URL traite la requête et renvoie une réponse.

Avec requests, ce n'est pas bien compliqué: pour récupérer des données, il suffit de faire:
```python
r = requests.get(url, paramètres)  # on a les 3 parties d'une requête HTTP: méthode, url, paramètres
```

Pour accéder au contenu de la réponse, il suffit de faire:

```python
r.json()  # si la réponse est en json, on peut ainsi récupérer un dicitonnaire
r.text    # pour récupérer la réponse au format texte
```

On disait tout à l'heure qu'une machine pouvait faire une requête sur un site humain normal. On accède ici à la page Wikipedia de l'artiste [Claire Fontaine](https://en.wikipedia.org/wiki/Claire_Fontaine).

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Claire_Fontaine")
print(r.url)
print(r.text)  # là, on voit pourquoi les API existent et ont des formats de réponse plus légers...

### En pratique: constituer le corpus

**L'API de Katabase** (documentation [ici](https://katabase.huma-num.fr/Katapi_documentation), code source [ici](https://github.com/katabase/Application)) sera utilisée pour constituer les corpus. L'API utilise une syntaxe simple et permet, entre autres, de récupérer toutes les entrées de catalogues associées au nom d'un.e auteur.ice, au format `json` ou sous la forme d'un document [`xml-tei`](https://fr.wikipedia.org/wiki/Text_Encoding_Initiative) complet (le standard pour l'encodage de documents textuels).

On fait une première requête pour voir la structure de la réponse (je ne rentre pas dans les détails, c'est le résultat qui importe ici)

In [ ]:
r = requests.get("https://katabase.huma-num.fr/katapi?level=item&name=S%C3%A9vign%C3%A9&sell_date=1800-1900&orig_date=1500-1800&format=json")
print(json.dumps(r.json(), indent=4))

Dans les grandes lignes, une réponse de `KatAPI` est faite de
- un en-tête `head` qui définit à la fois la requête et sa réponse
- un corps `results`, qui contient les résultats de la requête. Chaque résultat est représenté par un couple clé-valeur, où la clé est l'identifiant unique de l'entrée dans la base de données de Katabase et la valeur une représentation JSON de l'entrée de catalogue.

On définit donc une fonction qui, pour chaque nom d'auteur.ice dans nos corpus,
- lancera une requête sur l'API
- récupérera le résultat en JSON et le filtrera pour ne garder que les éléments nécessaires.
- l'ajoutera à un dictionnaire stockant tous les résultats pour un corpus.

In [ ]:
def katapi_request(dataset, author_name):
    """
    faire une requête sur l'API.
    
    structure de `dataset`, la variable de sortie:
    {
        "identifiant": {
            # description JSON du manuscrit
        }
    }
    
    :param author_name: le nom de l'auteur.ice à requêter
    :param dataset: le dictionnaire auquel rajouter les résultats de la requête.
    """
    # définir les paramètres
    root_url = " https://katabase.huma-num.fr/katapi?"  # l'URL pointant vers l'API
    params = {
        # les paramètres de notre requête
        "level": "item",  # la requête est faite sur les entrées de catalogue, pas sur un catalogue complet
        "format": "json",  # la réponse devra être en `json` (non en `xml-tei`)
        "sell_date": "1850-1910",  # les manuscrits ont été vendus entre 1850 et 1900
        "name": author_name  # l'auteur.ice du manuscrit est
    }
    
    # faire la requête
    r = requests.get(root_url, params=params)
    
    # l'url de la requête est construit tout seul par `requests`
    # à partir de `root_url` et `params`:
    print(r.url)
    
    # récupérer la réponse en json
    response = r.json()
    
    # on ajoute les résultats à notre variable de sortie
    for manuscript_id in response["results"]:
        dataset[manuscript_id] = response["results"][manuscript_id]
        
    return dataset


# on utilise ensuite cette fonction pour construire 4 jeux
# de données, un par entrée. à chaque itération, on ajoute
# à la variable `data_...` les résultats retournés par l'API
# pour un nouveau nom
for auteur in auteurs_idees:
    data_idees = katapi_request(data_idees, auteur)
for auteur in auteurs_theatre:
    data_theatre = katapi_request(data_theatre, auteur)
for auteur in auteurs_roman:
    data_roman = katapi_request(data_roman, auteur)
for auteur in auteurs_poeme:
    data_poeme = katapi_request(data_poeme, auteur)


---

## Du JSON au corpus en texte brut

Notre jeu de données de base est constitué. Il est stocké dans 4 variables, une corpus/genre. Il reste encore 3 étapes:
- nettoyer les jeux de données
- les représenter sous la forme de texte brut
- les enregistrer en fichiers texte.

### Nettoyer les réponses

On écrit une fonction pour nettoyer un jeu de données.

In [ ]:
def clean_dataset(dataset):
    """
    ici, on traite légèrement les jeux de données en JSON:
    - on supprime les éléments de description des entrées qui ne
      font pas partie du texte source (à l'exception du prix en 
      francs constants) et qui ne nous serviront pas ensuite
    - on supprime les espaces inutiles et sauts de lignes à l'intérieur
      de la description
    
    :param dataset: un jeu de données au format JSON construit avec `get_katabase_dataset()`.
    :returns: le jeu de données nettoyé.
    """
    dataset_out = {}  # jeu de données de sortie
    
    # on itère sur chaque clé et valeur du jeu de données
    # pour nettoyer les valeurs
    for key, value in dataset.items():
        
        # on supprime les éléments inutiles
        value.pop("author_wikidata_id", None)  # l'identifiant wikidata
        value.pop("format", None)  # le format de l'autographe
        value.pop("term", None)  # un terme normalisé décrivant le type de manuscrit (lettre autographe...)
            
        # on utilise une expression régulière pour supprimer les grands vides dans
        # le JSON: sauts de ligne, indentations...
        # - `\n*`: 0 à plusieurs sauts de ligne
        # - `\s+`: un ou plusieurs espaces
        # - on utilise `re.sub()` pour faire un remplacement. on remplace le motif ci-dessus par un espace.
        #   syntaxe: `re.sub(motif, remplacement, texte)`
        value["desc"] = re.sub("\n*\s+", " ", value["desc"])
        
        # on ajoute l'entrée nettoyée à notre jeu de données
        dataset_out[key] = value
    
    return dataset_out

# nettoyer les jeux de données
data_idees = clean_dataset(data_idees)
data_theatre = clean_dataset(data_theatre)
data_roman = clean_dataset(data_roman)
data_poeme = clean_dataset(data_poeme)

print(data_poeme)

### Transformer les dictionnaires en texte brut

Pour le moment, les entrées de catalogue sont représentées sous la forme d'un dictionnaire structuré. Or, tout l'enjeu du tutoriel sur la fouille de texte était de détecter des motifs à partir d'un texte brut. Donc, doit écrire une fonction pour représenter les jeux de données sous la forme de texte brut. Pour rappel, la structure attendue est la suivante: 

```
Nom de l'auteur.ice
Écrit en [date]. Vendu en [date].
Dimensions: [nombre décimal].
[description en texte libre]
Prix: [nombre décimal] [monnaie] (en francs constants 1900: [nombre décimal].  # ligne optionnelle
[saut de ligne]
[saut de ligne]
```

In [ ]:
def make_text(dataset):
    """
    ici, on construit un fichier en texte brut à partir du JSON récupéré
    par l'API Katabase et on l'enregistre dans un fichier.
    
    :param dataset: le jeu de données à partir duquel produire un texte brut.
    """
    text_out = ""  # le texte brut qui sera produit
    
    # on utilise `.values()` pour itérer seulement sur les valeurs: 
    # - `.values()` produit une liste des valeurs d'un dictionnaire,
    #   c'est-à-dire des éléments à droite dans des entrées de
    #   dictionnaire
    # - l'identifiant des entrées de catalogues ne sera pas retenu 
    #   dans le texte qu'on est en train de construire, donc pas la
    #   peine d'itérer sur celui-ci.
    for value in dataset.values():
        value_to_string = ""  # la version texte de l'entrée de catalogue
        
        # on ajoute le nom de l'auteur.
        # la syntaxe `f""` permet d'intégrer la valeur
        # de variables dans du texte: 
        # les variables s'écrivent entre `{}`. leur valeur est évaluée
        # et rajoutée à l'intérieur de la chaîne de caractère.
        value_to_string = f"{value['author']}\n"
        
        # si il existe une date d'écriture du manuscrit, on l'ajoute
        # `+=` permet d'incrémenter une variable: on ajoute à une
        # variable la valeur à droite
        if value["date"] is not None:
            value_to_string += f"Écrit en {value['date']}. "
        
        # on ajoute ensuite la date de vente, le nombre de pages et une description
        # l'opérateur `+` permet de concaténer du texte, c'est à dire de mettre plusieurs
        # bouts de texte bout à bout
        value_to_string += f"Vendu en {value['sell_date']}.\n"\
                           + f"Dimensions: {value['number_of_pages']} pages.\n"\
                           + f"{value['desc']}\n"
        
        # si il y a un prix, on ajoute le prix, la monnaie et le prix en francs constants.
        if value["price"] is not None:
            value_to_string += f"Prix: {value['price']} {value['currency']} "\
                               + f"(en francs constants 1900: {value['price_c']}).\n"
        
        # enfin, pour signifier la fin d'une entrée, on ajoute 2 lignes vides
        value_to_string += "\n\n"
        
        # on ajoute cette entrée à `text_out`, pour créer notre document de sortie.
        text_out += value_to_string
    
    return text_out

# maintenant on transforme en texte nos 4 corpus.
corpus_theatre = make_text(data_theatre)
corpus_idees = make_text(data_idees)
corpus_roman = make_text(data_roman)
corpus_poeme = make_text(data_poeme)

print(corpus_theatre)

### Enregistrer les fichiers

Enfin, on enregistre les 4 corpus produits dans des fichiers. La syntaxe pour enregistrer un fichier est la même que celle pour lire un fichier vue dans le notebook précédent: 

```python
with open(nom_du_fichier, mode="w") as fh:
    fh.write("texte à écrire")
```

Les deux seules choses qui changent sont:
- le `mode` dans lequel le fichier est ouvert avec `open()`: on met `w`, pour `write`, en mode de `open()`, ce qui signifie que le fichier est ouvert en écriture.
- on utilise `fh.write()` pour écrire (avant, on utilisait `fh.read()` pour lire.

In [ ]:
# on  crée une fonction pour écrire les résultats dans un fichier
def write_corpus(corpus, genre):
    """
    écrire le résultat dans un fichier.
    :param corpus: le texte à écrire
    :param genre: le genre du corpus, pour nommer le fichier comme il faut
    """
    outpath = os.path.join(f"catalogue_{genre}.txt")
    with open(outpath, mode="w+") as fh:
        fh.write(corpus)  # on y écrit le contenu de `corpus`

# enfin on écrit nos fichiers
write_corpus(corpus_roman, "roman")
write_corpus(corpus_idees, "idees")
write_corpus(corpus_poeme, "poeme")
write_corpus(corpus_theatre, "theatre")

## Et voilà !

En 2 temps 3 mouvements (ou presque), on a découvert ce que c'était une API et constitué 4 petits  corpus de recherche. On les a également enregistrés pour pouvoir les réutiliser plus tard. Ce qu'on a vu là n'était qu'une petite introduction aux API, qui vise à vous donner des outils pour votre recherche plus tard. Même si les API n'ont pas l'air phénoménalement utiles là tout de suite, c'est en continuant de les utiliser qu'on découvre tout leur potentiel et leur utilité pour la recherche en humanités : )

## [Notebook suivant](https://colab.research.google.com/drive/1DIM-zHq230x6fyZRoKOuYIf3K7udxloB?usp=sharing)